In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from deap import base, creator, tools, algorithms
import random

# === Load dataset ===
df = pd.read_csv("/dataset/balanced100k_2018.csv")


<ipython-input-3-ec8dd07c012f>:9: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/network/balanced100k_2018.csv")


In [ ]:
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Flow ID,Src IP,Src Port,Dst IP
0,80,6,20/02/2018 10:33:40,1172999,3,4,20.0,964.0,20.0,0.0,...,0.0,0.0,0.0,0.0,0.0,DDoS attacks-LOIC-HTTP,172.31.69.25-52.14.136.135-80-60344-6,52.14.136.135,60344.0,172.31.69.25
1,52302,6,20/02/2018 01:29:08,2378,3,1,31.0,0.0,31.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Benign,172.31.65.6-23.36.33.160-52302-443-6,23.36.33.160,443.0,172.31.65.6
2,80,6,16/02/2018 01:46:10,9021,2,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk,NaN,NaN,NaN,NaN
3,443,6,20/02/2018 03:32:39,59834166,9,15,548.0,9245.0,183.0,0.0,...,86331.0,19844513.333333,9559292.589311,25480312.0,8807222.0,Benign,152.195.32.112-172.31.66.19-443-51217-6,172.31.66.19,51217.0,152.195.32.112
4,8080,6,02/03/2018 02:15:12,10819,3,4,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599992,8080,6,02/03/2018 02:26:40,10952,3,4,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN
599993,8080,6,02/03/2018 12:06:49,499,2,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN
599994,443,6,20/02/2018 12:36:32,77952,2,1,74.0,126.0,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Benign,172.31.69.12-13.89.188.5-49672-443-6,172.31.69.12,49672.0,13.89.188.5
599995,8080,6,02/03/2018 11:18:17,13578,3,4,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN


In [ ]:
df.drop(labels=[ 'Flow ID', 'Src IP', 'Src Port',
              'Dst IP', 'Dst Port', 'Protocol', 'Timestamp'], axis=1,inplace=True)
df = df[df['Flow Duration'] != 'Flow Duration']



In [ ]:
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Flow ID,Src IP,Src Port,Dst IP
0,80,6,20/02/2018 10:33:40,1172999,3,4,20.0,964.0,20.0,0.0,...,0.0,0.0,0.0,0.0,0.0,DDoS attacks-LOIC-HTTP,172.31.69.25-52.14.136.135-80-60344-6,52.14.136.135,60344.0,172.31.69.25
1,52302,6,20/02/2018 01:29:08,2378,3,1,31.0,0.0,31.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Benign,172.31.65.6-23.36.33.160-52302-443-6,23.36.33.160,443.0,172.31.65.6
2,80,6,16/02/2018 01:46:10,9021,2,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,DoS attacks-Hulk,NaN,NaN,NaN,NaN
3,443,6,20/02/2018 03:32:39,59834166,9,15,548.0,9245.0,183.0,0.0,...,86331.0,19844513.333333,9559292.589311,25480312.0,8807222.0,Benign,152.195.32.112-172.31.66.19-443-51217-6,172.31.66.19,51217.0,152.195.32.112
4,8080,6,02/03/2018 02:15:12,10819,3,4,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599992,8080,6,02/03/2018 02:26:40,10952,3,4,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN
599993,8080,6,02/03/2018 12:06:49,499,2,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN
599994,443,6,20/02/2018 12:36:32,77952,2,1,74.0,126.0,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Benign,172.31.69.12-13.89.188.5-49672-443-6,172.31.69.12,49672.0,13.89.188.5
599995,8080,6,02/03/2018 11:18:17,13578,3,4,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Bot,NaN,NaN,NaN,NaN


In [ ]:

# if df["Label"].dtype == object:
#     df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

ALL_FEATURES = df.columns.tolist()
ALL_FEATURES.remove("Label")


In [ ]:
df

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1172999,3,4,20.0,964.0,20.0,0.0,6.666667,11.547005,964.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,2378,3,1,31.0,0.0,31.0,0.0,10.333333,17.897858,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,9021,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,59834166,9,15,548.0,9245.0,183.0,0.0,60.888889,91.33379,1460.0,...,20,100208.666667,16720.383498,118772.0,86331.0,19844513.333333,9559292.589311,25480312.0,8807222.0,1
4,10819,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599992,10952,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
599993,499,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
599994,77952,2,1,74.0,126.0,74.0,0.0,37.0,52.325902,126.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
599995,13578,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
df[ALL_FEATURES] = df[ALL_FEATURES].apply(pd.to_numeric, errors='coerce')
df[ALL_FEATURES] = df[ALL_FEATURES].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=ALL_FEATURES)  # drop rows with NaN in any feature


In [ ]:
df

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1172999,3,4,20.0,964.0,20.0,0.0,6.666667,11.547005,964.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
1,2378,3,1,31.0,0.0,31.0,0.0,10.333333,17.897858,0.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
2,9021,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,32,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
3,59834166,9,15,548.0,9245.0,183.0,0.0,60.888889,91.333790,1460.0,...,20,100208.666667,16720.383498,118772.0,86331.0,1.984451e+07,9.559293e+06,25480312.0,8807222.0,1
4,10819,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599992,10952,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
599993,499,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
599994,77952,2,1,74.0,126.0,74.0,0.0,37.000000,52.325902,126.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1
599995,13578,3,4,326.0,129.0,326.0,0.0,108.666667,188.216188,112.0,...,20,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1


In [ ]:
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(df[ALL_FEATURES])


In [ ]:
X_scaled

array([[9.77498350e-03, 1.07334142e-05, 1.64962059e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.98083337e-05, 1.07334142e-05, 4.12405147e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.51666679e-05, 5.36670710e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.49591677e-04, 5.36670710e-06, 4.12405147e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.13141669e-04, 1.07334142e-05, 1.64962059e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.41916669e-05, 5.36670710e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:

MUTATED_FEATURES = [

    'Fwd Pkt Len Mean',
    'Bwd Pkt Len Mean',
    'Pkt Len Mean',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Init Fwd Win Byts',
    'Init Bwd Win Byts'
]


In [ ]:
# =============================
# ✅ GA Mutation Using Custom Model with Scaled Inputs & Stable Bounds
# =============================


# === Scale features ===


X = X_scaled

X_original = df[ALL_FEATURES].values
y = df["Label"].values

# === Load pre-trained model ===
model = load_model("/content/drive/MyDrive/network/2018_100k_balanced_model_scaled.keras")

# === Select attack samples ===
attack_labels = [
    "Bot",
    "DDoS attacks-LOIC-HTTP",
    "DoS attacks-GoldenEye",
    "DoS attacks-Hulk",
    "DoS attacks-SlowHTTPTest",
    "DoS attacks-Slowloris",
    "FTP-BruteForce",
    "Infilteration",
    "SSH-Bruteforce"
]
MUT_IDX = [ALL_FEATURES.index(f) for f in MUTATED_FEATURES]

attack_indices = np.where(pd.Series(y).isin(attack_labels))[0]
X_attack = X[attack_indices]

# === Prediction function ===
ATTACK_CLASS_INDICES = [1,2,3,4,5,6,7,8,9]
def model_predict_prob_vector(sample):
    sample = np.array(sample).reshape(1, -1, 1)
    return model.predict(sample, verbose=0)[0]

def attack_probability(pred_vector):
    return sum(pred_vector[i] for i in ATTACK_CLASS_INDICES)

# === Fitness Function ===
def fitness_function(individual):
    full_sample = original_sample.copy()
    for i, idx in enumerate(MUT_IDX):
        full_sample[idx, 0] = individual[i]
    pred_vector = model_predict_prob_vector(full_sample)
    attack_prob = attack_probability(pred_vector)
    distortion = np.linalg.norm(np.array(individual) - original_sample[MUT_IDX, 0])
    return (1.0 - attack_prob, -distortion)

# === GA Preparation ===weights=(0.7, 1.3)
creator.create("FitnessMulti", base.Fitness, weights=(0.7, 1.3))
creator.create("Individual", list, fitness=creator.FitnessMulti)

def bounded_uniform(low, high, center, width=0.2):
    span = (high - low) * width
    return random.uniform(max(low, center - span), min(high, center + span))

def run_ga(original_sample, generations=10, population_size=30):
    toolbox = base.Toolbox()
    for i in range(len(MUT_IDX)):
        low, high = 0.0, 1.0
        center = original_sample[MUT_IDX[i], 0]
        toolbox.register(f"attr_float_{i}", bounded_uniform, low, high, center, 0.2)

    toolbox.register("individual", tools.initCycle, creator.Individual,
                     tuple(toolbox.__getattribute__(f"attr_float_{i}") for i in range(len(MUT_IDX))), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.05, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=population_size)

    for gen in range(generations):
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.6:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.3:
                toolbox.mutate(mutant)
                del mutant.fitness.values
                for i in range(len(mutant)):
                    mutant[i] = np.clip(mutant[i], 0.0, 1.0)

        invalid = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid:
            ind.fitness.values = toolbox.evaluate(ind)

        elite = tools.selBest(population, k=1)
        population[:] = offspring[:-1] + elite

    best = tools.selBest(population, k=1)[0]
    print(f"Gen {gen+1}: Best fitness = {best.fitness.values}")

    return best


# === Run GA for top 5 attack samples ===
# mutated_full = []
# for idx, row_idx in enumerate(attack_indices[1:3]):
#   print(row_idx)
# for idx, row_idx in enumerate(attack_indices[:5]):

#     print(f"\n🚀 Mutating GA sample {idx+1}/5")
#     original_sample = X[row_idx].copy()
#     best_ind = run_ga(original_sample)
#     full_sample = original_sample.copy()
#     for i, idx_feat in enumerate(MUT_IDX):
#         full_sample[idx_feat] = best_ind[i]
#     mutated_full.append(full_sample)

# # Save mutated adversarial samples
# mutated_full_df = pd.DataFrame(scaler.inverse_transform(mutated_full), columns=ALL_FEATURES)
# mutated_full_df["Label"] = 1
# mutated_full_df.to_csv("mutated_custom_model_samples.csv", index=False)
# print("✅ Adversarial samples using custom model saved.")


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
print(len(attack_indices))

499707


In [ ]:
statistics = []

for idx, row_idx in enumerate(attack_indices[51:150]):
    print(f"\n🚀 Mutating GA sample {idx+1}/5")

    original_sample = X_scaled[row_idx].reshape(-1, 1).copy()
    original_label = y[row_idx]

    pred_vector_before = model_predict_prob_vector(original_sample)
    attack_prob_before = attack_probability(pred_vector_before)
    predicted_label_before = np.argmax(pred_vector_before)

    best_ind = run_ga(original_sample)

    mutated_sample = original_sample.copy()
    for i, idx_feat in enumerate(MUT_IDX):
        mutated_sample[idx_feat, 0] = best_ind[i]

    pred_vector_after = model_predict_prob_vector(mutated_sample)
    attack_prob_after = attack_probability(pred_vector_after)
    predicted_label_after = np.argmax(pred_vector_after)

    mutation_count = np.sum(original_sample.flatten() != mutated_sample.flatten())
    feature_change_ratio = mutation_count / len(ALL_FEATURES)
    distortion = np.linalg.norm(mutated_sample.flatten() - original_sample.flatten())

    statistics.append({
        "sample_id": idx,
        "original_label": original_label,
        "predicted_label_before": predicted_label_before,
        "predicted_label_after": predicted_label_after,
        "attack_prob_before": attack_prob_before,
        "attack_prob_after": attack_prob_after,
        "distortion": distortion,
        "mutation_count": mutation_count,
        "feature_change_ratio": feature_change_ratio,
        "original_features": original_sample.flatten().tolist(),
        "mutated_features": mutated_sample.flatten().tolist(),
        "attack_class_prob_vector_before": pred_vector_before.tolist(),
        "attack_class_prob_vector_after": pred_vector_after.tolist()
    })

# Save statistics to CSV
# statistics_df = pd.DataFrame(statistics)
# statistics_df.to_csv("mutation_statistics.csv", index=False)

print("✅ All adversarial samples and mutation statistics saved!")



🚀 Mutating GA sample 1/5
Gen 10: Best fitness = (np.float32(0.99999994), np.float64(-0.31802806475208406))

🚀 Mutating GA sample 2/5
Gen 10: Best fitness = (np.float32(0.9999972), np.float64(-0.3614582403804275))

🚀 Mutating GA sample 3/5
Gen 10: Best fitness = (np.float32(0.80237705), np.float64(-0.4493141370389447))

🚀 Mutating GA sample 4/5
Gen 10: Best fitness = (np.float32(0.7161729), np.float64(-0.39184866054374917))

🚀 Mutating GA sample 5/5
Gen 10: Best fitness = (np.float32(0.69873154), np.float64(-0.4986520491814565))

🚀 Mutating GA sample 6/5
Gen 10: Best fitness = (np.float32(0.99999875), np.float64(-0.37650967289547577))

🚀 Mutating GA sample 7/5
Gen 10: Best fitness = (np.float32(0.97758394), np.float64(-0.32417030471603325))

🚀 Mutating GA sample 8/5
Gen 10: Best fitness = (np.float32(0.9797792), np.float64(-0.28292084885224605))

🚀 Mutating GA sample 9/5
Gen 10: Best fitness = (np.float32(0.9457901), np.float64(-0.29062195876513325))

🚀 Mutating GA sample 10/5
Gen 10: 

In [ ]:
statistics_df = pd.DataFrame(statistics)

statistics_df.to_csv("/content/drive/MyDrive/network/mutation_statistics_parameter adjust_50_100.csv", index=False,sep=";")
